In [ ]:
!pip install osmnx

In [ ]:
import networkx as nx
import osmnx as ox
import pandas as pd
import numpy as np
from collections import Counter
import math
%matplotlib inline

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Buildings

In [ ]:
#get all buildings in bounding box
G = ox.geometries_from_bbox(48.2494, 48.1572, 16.2975, 16.4380, tags = {'building': True})

/usr/local/lib/python3.7/dist-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/usr/local/lib/python3.7/dist-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [ ]:
G

In [ ]:
#only keep polygon geometries
buildings = G.loc[G.geometry.geometry.type=='Polygon']

In [ ]:
#only keep relevant columns
buildings = buildings[['height', 'geometry']]

In [ ]:
#if building does have a height, set height = 12
buildings['height'] = buildings['height'].fillna(12)

In [ ]:
#clean up height column so that it only contains numbers
disallowed_characters = "m"
heights_cleaned = []

for i in range(len(buildings.index)):
  name = list(buildings['height'])[i]
  try:
    for character in disallowed_characters:
	    heights_cleaned.append(name.replace(character, ""))
  except Exception:
    heights_cleaned.append(name)
   

In [ ]:
#drop original height column and add your cleaned up one
buildings = buildings.drop(columns = ['height'])
buildings['height'] = heights_cleaned

In [ ]:
#coonvert all height values to floats
buildings['height'] = buildings['height'].astype(float)

In [ ]:
#check to see if height values are clean
set(list(buildings['height']))

In [ ]:
# Reproject to a projected crs (meters)
crs = 3857

buildings_proj = ox.projection.project_gdf(buildings, to_crs=crs)

In [ ]:
buildings_proj

geometry  \
element_type osmid                                                         
way          5887599   POLYGON ((1823504.819 6138752.075, 1823505.365...   
             8083399   POLYGON ((1821295.517 6141940.658, 1821308.297...   
             8097149   POLYGON ((1822448.564 6139913.816, 1822453.518...   
             8097153   POLYGON ((1822666.450 6140013.788, 1822653.593...   
             8097556   POLYGON ((1820974.817 6140853.051, 1820913.046...   
...                                                                  ...   
relation     13817690  POLYGON ((1815815.993 6134624.456, 1815821.192...   
             13872064  POLYGON ((1821523.221 6141904.325, 1821519.102...   
             13872065  POLYGON ((1821479.695 6141933.842, 1821480.820...   
             13928463  POLYGON ((1823918.772 6139900.723, 1823946.780...   
             14050714  POLYGON ((1822632.008 6134588.922, 1822631.507...   

                       height  
element_type osmid             
way          5887599     12.0  
             8083399     12.0  
             8097149     22.0  
             8097153     12.0  
             8097556     25.0  
...                       ...  
relation     13817690    12.0  
             13872064    24.5  
             13872065    29.5  
             13928463    12.0  
             14050714    12.0  

[79618 rows x 2 columns]

In [ ]:
#write to geojson file
buildings_proj.to_file('/content/drive/MyDrive/S0_STUDIO/AIA_Studio/Data/infraredWind/buildings_proj.geojson', driver="GeoJSON") 

### Streets

In [ ]:
S = ox.graph_from_bbox(48.2494, 48.1572, 16.2975, 16.4380, simplify = True)

In [ ]:
S_nodes, S_edges = ox.graph_to_gdfs(S)

In [ ]:
streets = S_edges[['osmid', 'geometry']]

In [ ]:
# Reproject to a projected crs (meters)
crs = 3857
streets_proj = ox.projection.project_gdf(streets, to_crs=crs)

In [ ]:
streets_proj.loc[(199732, 2447309494, 0)]['osmid']

29065909

In [ ]:
#street segments need to be exploded so that it can be written to a geojson
#get the intersection of the linestring geometry
ints = streets_proj.intersection(streets_proj)

In [ ]:
#explode at intersections
explode = ints.explode()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  """Entry point for launching an IPython kernel.


In [ ]:
explode

u           v           key   
199732      2447309494  0    0    LINESTRING (1821202.064 6141639.210, 1821205.2...
            4683383695  0    0    LINESTRING (1821202.064 6141639.210, 1821207.4...
                             1    LINESTRING (1821207.486 6141630.507, 1821209.1...
                             2    LINESTRING (1821209.156 6141627.835, 1821217.1...
            318697385   0    0    LINESTRING (1821202.064 6141639.210, 1821192.5...
                                                        ...                        
9758085574  1846835856  0    0    LINESTRING (1821861.154 6138121.680, 1821800.7...
            1846835874  0    0    LINESTRING (1821861.154 6138121.680, 1821911.3...
            9758085573  0    0    LINESTRING (1821861.154 6138121.680, 1821855.3...
            9758085575  0    0    LINESTRING (1821861.154 6138121.680, 1821881.7...
9758085575  9758085574  0    0    LINESTRING (1821881.792 6138093.646, 1821861.1...
Length: 325859, dtype: geometry

In [ ]:
import geopandas as gp

In [ ]:
#make new geopandas data frame with explode street edges
streets_explode = gp.GeoDataFrame({'geometry':explode})
streets_explode

geometry
u          v          key                                                     
199732     2447309494 0   0  LINESTRING (1821202.064 6141639.210, 1821205.2...
           4683383695 0   0  LINESTRING (1821202.064 6141639.210, 1821207.4...
                          1  LINESTRING (1821207.486 6141630.507, 1821209.1...
                          2  LINESTRING (1821209.156 6141627.835, 1821217.1...
           318697385  0   0  LINESTRING (1821202.064 6141639.210, 1821192.5...
...                                                                        ...
9758085574 1846835856 0   0  LINESTRING (1821861.154 6138121.680, 1821800.7...
           1846835874 0   0  LINESTRING (1821861.154 6138121.680, 1821911.3...
           9758085573 0   0  LINESTRING (1821861.154 6138121.680, 1821855.3...
           9758085575 0   0  LINESTRING (1821861.154 6138121.680, 1821881.7...
9758085575 9758085574 0   0  LINESTRING (1821881.792 6138093.646, 1821861.1...

[325859 rows x 1 columns]

In [ ]:
streets_explode2 = streets_proj.explode()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  """Entry point for launching an IPython kernel.


In [ ]:
osmid = []
for i in range(len(streets_explode.index)):
  osmid.append(streets_proj.loc[(streets_explode.index[i][0], streets_explode.index[i][1], streets_explode.index[i][2])]['osmid'])


In [ ]:
streets_explode

geometry
u          v          key                                                     
199732     2447309494 0   0  LINESTRING (1821202.064 6141639.210, 1821205.2...
           4683383695 0   0  LINESTRING (1821202.064 6141639.210, 1821207.4...
                          1  LINESTRING (1821207.486 6141630.507, 1821209.1...
                          2  LINESTRING (1821209.156 6141627.835, 1821217.1...
           318697385  0   0  LINESTRING (1821202.064 6141639.210, 1821192.5...
...                                                                        ...
9758085574 1846835856 0   0  LINESTRING (1821861.154 6138121.680, 1821800.7...
           1846835874 0   0  LINESTRING (1821861.154 6138121.680, 1821911.3...
           9758085573 0   0  LINESTRING (1821861.154 6138121.680, 1821855.3...
           9758085575 0   0  LINESTRING (1821861.154 6138121.680, 1821881.7...
9758085575 9758085574 0   0  LINESTRING (1821881.792 6138093.646, 1821861.1...

[325859 rows x 1 columns]

In [ ]:
type(osmid[50])

list

In [ ]:
count = 0
for id in osmid:
  if type(id) == 'list':
    length = len(id)
    for value in range(0,length):
      osmid[i+value] = osmid[i][value]
      count+=1

In [ ]:
#write to geojson file
streets_explode.to_file('/content/drive/MyDrive/S0_STUDIO/AIA_Studio/Data/infraredWind/streets_proj.geojson', driver="GeoJSON") 